In [880]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import talib
from math import floor

In [310]:
def open_ticker_csv(ticker_name: str, timeframe: str, directory: str) -> pd.DataFrame:
    ticker_name = '-'.join(ticker_name.split('/'))
    ticker_name = f'{ticker_name}_{timeframe}' if timeframe else f'{ticker_name}'
    ticker_path = f'{directory}/{ticker_name}.csv'
    columns_names = ('timestamp', 'open', 'high', 'low', 'close', 'volume')

    ticker = pd.read_csv(ticker_path,
                         index_col=0,
                         header=None,
                         names=columns_names)

    return ticker

In [311]:
def clean_ticker(ticker: pd.DataFrame) -> pd.DataFrame:
    ticker_copy = ticker.copy()
    ticker_copy.index = pd.to_datetime(ticker_copy.index, unit='ms')
    ticker_copy = ticker_copy.drop_duplicates().sort_index()

    return ticker_copy

In [312]:
def reindex_tickers(tickers: dict) -> dict:
    combined_ticker_index = None
    reindexed_tickers = dict()

    for ticker in tickers.values():
        if combined_ticker_index is None:
            combined_ticker_index = ticker.index
        else:
            combined_ticker_index = combined_ticker_index.union(ticker.index)

    for ticker, data in tickers.items():
        data = data.reindex(index=combined_ticker_index,
                            method='pad',
                            fill_value=np.nan)
        reindexed_tickers[ticker] = data

    return reindexed_tickers

In [313]:
def find_pos_price(price, price_set):
    return len(price_set) - np.where(price_set == price)[0][0]

In [897]:
def BULL_DIV(price_set, rsi_set, price_pk_prominence, window=1):
    from scipy.signal import find_peaks

    peaks_price, _ = find_peaks(-price_set, prominence=price_pk_prominence)
    ref_price = [price_set[item]
                 for item in peaks_price]
    ref_rsi = [rsi_set[item]
               for item in peaks_price]
    
    print(ref_price)

    counter = 0
    for i in range(len(ref_price)-1, 0, -1):
        if price_set[-1] < ref_price[i] and rsi_set[-1] > ref_rsi[i]:
            counter += 1
            latest_peak_price = ref_price[i]
            latest_peak_rsi = ref_rsi[i]

        if counter == window:
            return {
                'is_div': True,
                'price_peaks': (price_set[-1], latest_peak_price),
                'rsi_peaks': (rsi_set[-1], latest_peak_rsi)
            }
    return False

def BULL_DIV_RSI(price_set, rsi_set, rsi_pk_prominence, window=1):
    from scipy.signal import find_peaks

    peaks_rsi, _ = find_peaks(-rsi_set, prominence=rsi_pk_prominence)
    ref_price = [price_set[item]
                 for item in peaks_rsi]
    ref_rsi = [rsi_set[item]
               for item in peaks_rsi]
    
    print(ref_price)

    counter = 0
    for i in range(len(ref_price)-1, 0, -1):
        if price_set[-1] < ref_price[i] and rsi_set[-1] > ref_rsi[i]:
            counter += 1
            latest_peak_price = ref_price[i]
            latest_peak_rsi = ref_rsi[i]

        if counter == window:
            return {
                'is_div': True,
                'price_peaks': (price_set[-1], latest_peak_price),
                'rsi_peaks': (rsi_set[-1], latest_peak_rsi)
            }
    return False

def BULL_DIV_2(price_set, rsi_set, price_pk_prominence):
    from scipy.signal import find_peaks

    peaks_price, _ = find_peaks(-price_set, prominence=price_pk_prominence)
    ref_price = [price_set[item]
                 for item in peaks_price]
    ref_rsi = [rsi_set[item]
               for item in peaks_price]
    
    latest_price = price_set[-1]
    latest_rsi = rsi_set[-1]
    latest_peak_price = None
    latest_peak_rsi = None
    latest_peak_pos = None
    is_div = False
    
    local_min = None
    counter = 0
    
    print(ref_price)
    
    for i in range(len(ref_price)-1, 0, -1):
        if floor(latest_price) < floor(ref_price[i]) and floor(latest_rsi) > floor(ref_rsi[i]):
            latest_peak_price = ref_price[i]
            latest_peak_rsi = ref_rsi[i]
            latest_peak_pos = find_pos_price(latest_peak_price, price_set)
            counter += 1
            is_div = True
            
            for j in range(latest_peak_pos-1, 1, -1):
                if latest_price > price_set[-j]:
                    local_min = price_set[-j]
                    is_div = False
                    counter = 0
                    break            
    return {
        'is_div': is_div,
        'price_peaks': (price_set[-1], latest_peak_price),
        'rsi_peaks': (rsi_set[-1], latest_peak_rsi),
        'local_min': local_min,
        'nb_div': counter
    }

def BULL_DIV_RSI_2(price_set, rsi_set, price_pk_prominence=1):
    from scipy.signal import find_peaks

    peaks_price, _ = find_peaks(-rsi_set, prominence=price_pk_prominence)
    ref_price = [price_set[item]
                 for item in peaks_price]
    ref_rsi = [rsi_set[item]
               for item in peaks_price]
    
    latest_price = price_set[-1]
    latest_rsi = rsi_set[-1]
    latest_peak_price = None
    latest_peak_rsi = None
    latest_peak_pos = None
    is_div = False
    
    local_min = None
    counter = 0
    
    print(price_set)
    
    for i in range(len(ref_price)-1, 0, -1):
        if latest_price < ref_price[i] and latest_rsi > ref_rsi[i]:
            latest_peak_price = ref_price[i]
            latest_peak_rsi = ref_rsi[i]
            latest_peak_pos = find_pos_price(latest_peak_price, price_set)
            counter += 1
            is_div = True
            
            for j in range(latest_peak_pos-1, 1, -1):
                if latest_price > price_set[-j]:
                    local_min = price_set[-j]
                    is_div = False
                    counter = 0
                    break            
    return {
        'is_div': is_div,
        'price_peaks': (price_set[-1], latest_peak_price),
        'rsi_peaks': (rsi_set[-1], latest_peak_rsi),
        'local_min': local_min,
        'nb_div': counter
    }

In [898]:
data = open_ticker_csv('BTC/USDT', '15m', '../exchange_data')
data = clean_ticker(data)
start_date = '2017-01-14 00:00:00'
end_date = '2019-01-15 15:00:00'
mask = (data.index >= start_date) & (data.index <= end_date)
data_custom = data.loc[mask]
datetimes = data_custom.index
close = data_custom['close']
data_custom['rsi'] = talib.RSI(data_custom['close'])
price_set = data_custom[data_custom['open'] > data_custom['close']]['low'].to_numpy()
rsi_set = data_custom[data_custom['open'] > data_custom['close']]['rsi'].to_numpy()

/Users/redamakhoukh/.local/share/virtualenvs/promesse_trading_engine_v2-NEvk7s3j/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [899]:
BULL_DIV(price_set[-20:], rsi_set[-20:], 10, window=1)

[3578.07, 3598.84]


{'is_div': True,
 'price_peaks': (3580.0, 3598.84),
 'rsi_peaks': (41.04298992562801, 40.57527915450784)}

In [907]:
BULL_DIV_2(price_set[-20:], rsi_set[-20:], 15)

[3578.07]


{'is_div': False,
 'price_peaks': (3580.0, None),
 'rsi_peaks': (41.04298992562801, None),
 'local_min': None,
 'nb_div': 0}

In [888]:
BULL_DIV_RSI_2(price_set[-20:], rsi_set[-20:])

[7950.   7903.02 7910.   7861.   7862.01 7875.01 7817.01 7811.   7836.01
 7818.84 7797.46 7710.   7852.69 7886.56 7837.56 7817.   7817.86 7870.78
 7857.14 7784.57]


{'is_div': False,
 'price_peaks': (7784.57, 7811.0),
 'rsi_peaks': (43.97520410574324, 35.89088112637818),
 'local_min': 7710.0,
 'nb_div': 0}

Timestamp('2019-05-17 06:00:00')

In [166]:
np.where(price_set[-20:] == 7150.33)[0][0]

15

In [169]:
len(price_set[-20:]) - np.where(price_set[-20:] == 7150.33)[0][0]

5

In [172]:
def find_pos_price(price, price_set):
    return len(price_set) - np.where(price_set == price)[0][0]

In [173]:
find_pos_price(7150.33, price_set[-20:])

5

In [478]:
data_custom

,open,high,low,close,volume,rsi
timestamp,,,,,,
2018-09-14 00:00:00,6487.39,6501.24,6454.68,6461.76,1105.991892,NaN
2018-09-14 01:00:00,6462.79,6495.09,6460.16,6488.99,896.648992,NaN
2018-09-14 02:00:00,6488.99,6565.68,6488.77,6551.00,2592.849598,NaN
2018-09-14 03:00:00,6551.00,6584.99,6542.13,6542.37,1584.898259,NaN
2018-09-14 04:00:00,6545.79,6570.00,6535.41,6545.00,1370.341288,NaN
...,...,...,...,...,...,...
2018-11-20 17:00:00,4854.14,4870.47,4631.00,4640.33,5903.448523,39.112416
2018-11-20 18:00:00,4644.73,4745.34,4635.05,4729.51,3396.254230,43.827191
2018-11-20 19:00:00,4727.90,4769.49,4651.32,4673.93,2228.087833,41.661943


In [11]:
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=
           [[{"type": "scatter"}],
           [{"type": "scatter"}]]
)

In [12]:
fig.add_trace(
    go.Scatter(
        x=datetimes,
        y=close,
        mode="lines",
        name="Closes",
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=datetimes,
        y=rsi,
        mode="lines",
        name="RSI"
    ),
    row=2, col=1
)


fig.update_layout(
    height=800,
    showlegend=False,
    title_text="BTC/USDT",
)

fig.show()

NameError: name 'rsi' is not defined

In [188]:
ls =  [0, 1, 2, 3, 4, 5]

In [196]:
for i in range(len(ls)-2, 0, -1):
    print(ls[i])

4
3
2
1


In [302]:
test = {'hi': 123}

In [308]:
test.keys()

dict_keys(['hi'])

In [466]:
def test():
    v = None
    for i in range(3):
        if True:
            if True:
                v = 1
    print(v)

In [467]:
test()

1
